# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [1]:
import torch as th

In [ ]:
x = th.tensor([1,2,3,4,5])
x

In [ ]:
y = x + x

In [ ]:
print(y)

In [2]:
import syft as sy

W0706 22:29:58.866479 140124961441536 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/home/raj/mooc/private-ai/.venv/lib/python3.6/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0706 22:29:58.877974 140124961441536 deprecation_wrapper.py:119] From /home/raj/mooc/private-ai/.venv/lib/python3.6/site-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [3]:
hook = sy.TorchHook(th)

In [ ]:
th.tensor([1,2,3,4,5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [ ]:
bob = sy.VirtualWorker(hook, id="bob")

In [ ]:
bob._objects

In [ ]:
x = th.tensor([1,2,3,4,5])

In [ ]:
x = x.send(bob)

In [ ]:
bob._objects

In [ ]:
x.location

In [ ]:
x.id_at_location

In [ ]:
x.id

In [ ]:
x.owner

In [ ]:
hook.local_worker

In [ ]:
x

In [ ]:
x = x.get()
x

In [ ]:
bob._objects

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [ ]:
# try this project here!

In [4]:
bob = sy.VirtualWorker(hook, id='bob')
alice = sy.VirtualWorker(hook, id='alice')

In [ ]:
print(bob._objects)
print(alice._objects)

In [ ]:
x = th.Tensor([1,2,3,4,5])

In [ ]:
x_ptr=x.send(alice,bob)

In [ ]:
x_ptr

In [ ]:
x_ptr.child

In [ ]:
print(bob._objects)
print(alice._objects)

In [ ]:
#x_ptr.get(sum_results=True)
x_ptr.get()

In [ ]:
print(bob._objects)
print(alice._objects)

# Lesson: Introducing Remote Arithmetic

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [ ]:
x

In [ ]:
y

In [ ]:
z = x + y

In [ ]:
z

In [ ]:
z = z.get()
z

In [ ]:
z = th.add(x,y)
z

In [ ]:
z = z.get()
z

In [ ]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [ ]:
z = (x + y).sum()

In [ ]:
z.backward()

In [ ]:
x = x.get()

In [ ]:
x

In [ ]:
x.grad

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [ ]:
# try this project here!

In [ ]:
import torch.nn as nn
from torch.nn.modules import Linear
from torch.nn.functional import relu, softmax

In [ ]:
class Model(nn.Module):
    def __init__(self, input_nodes=100, hidden_nodes=50, output_nodes=10):
        super(Model, self).__init__()
        self.linear1 = Linear(input_nodes, hidden_nodes)
        self.linear2 = Linear(hidden_nodes, output_nodes)
    
    def forward(self, x):
        x = relu(self.linear1(x))
        x = softmax(self.linear2(x))
        return x

In [ ]:
model = Model()

In [ ]:
model = model.send(bob)

In [ ]:
bob._objects

In [ ]:
input = th.rand([1000,100]).send(bob)
target = th.randint(1,[1000,10]).type(th.FloatTensor).send(bob)

In [ ]:
output = model(input)
loss = ((target-output)**2).sum()

# loss.get()


loss.backward()

model.linear1.weight.data += (model.linear1.weight.grad * 0.01)
model.linear2.weight.data += (model.linear2.weight.grad * 0.01)

# model.linear1.weight.grad.data *= 0
# model.linear2.weight.grad.data *= 0

print(loss.get())

# Lesson: Garbage Collection and Common Errors


In [ ]:
bob = bob.clear_objects()

In [ ]:
bob._objects

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
bob._objects

In [ ]:
del x

In [ ]:
bob._objects

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
bob._objects

In [ ]:
x = "asdf"

In [ ]:
bob._objects

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
x

In [ ]:
bob._objects

In [ ]:
x = "asdf"

In [ ]:
bob._objects

In [ ]:
del x

In [ ]:
bob._objects

In [ ]:
bob = bob.clear_objects()
bob._objects

In [ ]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
bob._objects

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [ ]:
z = x + y

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [ ]:
z = x + y

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [11]:
from torch import nn, optim

In [ ]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [ ]:
# A Toy Model
model = nn.Linear(2,1)

In [ ]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [ ]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

In [6]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

bob.clear_objects()
alice.clear_objects()

data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

print(bob._objects)

data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

print(alice._objects)

{96823205773: tensor([[1., 1.],
        [0., 1.]], requires_grad=True), 23592253519: tensor([[1.],
        [1.]], requires_grad=True)}
{47635199064: tensor([[1., 0.],
        [0., 0.]], requires_grad=True), 469430043: tensor([[0.],
        [0.]], requires_grad=True)}


In [7]:
alice._objects

{47635199064: tensor([[1., 0.],
         [0., 0.]], requires_grad=True), 469430043: tensor([[0.],
         [0.]], requires_grad=True)}

In [8]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [12]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)
#             print(alice._objects)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [13]:
train()

tensor(0.1285, requires_grad=True)
tensor(0.3544, requires_grad=True)
tensor(0.1450, requires_grad=True)
tensor(0.2457, requires_grad=True)
tensor(0.1139, requires_grad=True)
tensor(0.1791, requires_grad=True)
tensor(0.0854, requires_grad=True)
tensor(0.1314, requires_grad=True)
tensor(0.0640, requires_grad=True)
tensor(0.0967, requires_grad=True)
tensor(0.0480, requires_grad=True)
tensor(0.0714, requires_grad=True)
tensor(0.0360, requires_grad=True)
tensor(0.0528, requires_grad=True)
tensor(0.0270, requires_grad=True)
tensor(0.0391, requires_grad=True)
tensor(0.0202, requires_grad=True)
tensor(0.0290, requires_grad=True)
tensor(0.0152, requires_grad=True)
tensor(0.0216, requires_grad=True)
tensor(0.0114, requires_grad=True)
tensor(0.0160, requires_grad=True)
tensor(0.0086, requires_grad=True)
tensor(0.0119, requires_grad=True)
tensor(0.0064, requires_grad=True)
tensor(0.0089, requires_grad=True)
tensor(0.0048, requires_grad=True)
tensor(0.0066, requires_grad=True)
tensor(0.0036, requi

In [14]:
bob._objects

{96823205773: tensor([[1., 1.],
         [0., 1.]], requires_grad=True), 23592253519: tensor([[1.],
         [1.]], requires_grad=True)}

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [15]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [16]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [17]:
x = x.send(alice)

In [18]:
bob._objects

{43284942183: tensor([1, 2, 3, 4, 5])}

In [19]:
alice._objects

{11716068774: (Wrapper)>[PointerTensor | alice:11716068774 -> bob:43284942183]}

In [20]:
y = x + x

In [21]:
y

(Wrapper)>[PointerTensor | me:90197253150 -> alice:11096896228]

In [22]:
bob._objects

{43284942183: tensor([1, 2, 3, 4, 5]),
 84271354217: tensor([ 2,  4,  6,  8, 10])}

In [23]:
alice._objects

{11716068774: (Wrapper)>[PointerTensor | alice:11716068774 -> bob:43284942183],
 11096896228: (Wrapper)>[PointerTensor | alice:11096896228 -> bob:84271354217]}

In [24]:
jon = sy.VirtualWorker(hook, id="jon")

In [25]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [26]:
bob._objects

{78977307140: tensor([1, 2, 3, 4, 5])}

In [27]:
alice._objects

{1572026254: (Wrapper)>[PointerTensor | alice:1572026254 -> bob:78977307140]}

In [28]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:1572026254 -> bob:78977307140]

In [29]:
bob._objects

{78977307140: tensor([1, 2, 3, 4, 5])}

In [30]:
alice._objects

{}

In [31]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [32]:
bob._objects

{}

In [33]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [34]:
bob._objects

{22132445526: tensor([1, 2, 3, 4, 5])}

In [35]:
alice._objects

{13570789019: (Wrapper)>[PointerTensor | alice:13570789019 -> bob:22132445526]}

In [36]:
del x

In [37]:
bob._objects

{}

In [38]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [39]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [40]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [41]:
bob._objects

{97070379740: tensor([1, 2, 3, 4, 5])}

In [42]:
alice._objects

{}

In [43]:
x.move(alice)

(Wrapper)>[PointerTensor | me:93136731256 -> alice:93136731256]

In [44]:
bob._objects

{}

In [45]:
alice._objects

{93136731256: tensor([1, 2, 3, 4, 5])}

In [46]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [47]:
bob._objects

{36825933774: tensor([1, 2, 3, 4, 5])}

In [48]:
alice._objects

{93136731256: tensor([1, 2, 3, 4, 5]),
 92097031383: (Wrapper)>[PointerTensor | alice:92097031383 -> bob:36825933774]}

In [49]:
x.remote_get()

(Wrapper)>[PointerTensor | me:58451659356 -> alice:92097031383]

In [50]:
bob._objects

{}

In [51]:
alice._objects

{93136731256: tensor([1, 2, 3, 4, 5]), 92097031383: tensor([1, 2, 3, 4, 5])}

In [52]:
x.move(bob)

(Wrapper)>[PointerTensor | me:58451659356 -> bob:58451659356]

In [53]:
x

(Wrapper)>[PointerTensor | me:58451659356 -> bob:58451659356]

In [54]:
bob._objects

{58451659356: tensor([1, 2, 3, 4, 5])}

In [55]:
alice._objects

{93136731256: tensor([1, 2, 3, 4, 5])}